# Implementation
Author: Jørgen S. Dokken
## Creating the mesh
To create the computational geometry, we use the python-API of [GMSH](http://gmsh.info/). We start by import the gmsh-module, and initalizing it.

In [1]:
import gmsh
gmsh.initialize()

The next step is to create the membrane and starting the computations by the GMSH CAD kernel, to generate the relevant underlying data structures. The arguments into `addDisk` is the x, y and z coordinate of the center of the circle, while the to last argument is the x-radius and y-radius.

In [2]:
membrane = gmsh.model.occ.addDisk(0, 0, 0, 1, 1)
gmsh.model.occ.synchronize()

The next step is to make the membrane a physical surface, such that it is recognized by gmsh when generating the mesh. As a surface is a two-dimensional entity, we add two as the first argument, and the entity tag of the as the second argument. The last argument is the physical tag. In a later demo, we will get into when this tag matters.

In [3]:
gdim = 2
status = gmsh.model.addPhysicalGroup(gdim, [membrane], 1)

Finally, we generate the two-dimensional mesh. We set a uniform mesh size by modifying the GMSH options

In [4]:
gmsh.option.setNumber("Mesh.CharacteristicLengthMin",0.05)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax",0.05)
gmsh.model.mesh.generate(gdim)

We will import the GMSH-mesh directly from GMSH, using the approach in Section 2 of [A GMSH tutorial for dolfinx](http://jsdokken.com/converted_files/tutorial_gmsh.html). To make sure this runs in parallel and  serial, we will read in the mesh on one processor, and let dolfin-X distribute the mesh data among the processros.

In [5]:
from dolfinx.io import extract_gmsh_geometry,extract_gmsh_topology_and_markers, ufl_mesh_from_gmsh
from mpi4py import MPI
from warnings import filterwarnings

filterwarnings("ignore")
if MPI.COMM_WORLD.rank == 0:
    # Get mesh geometry
    geometry_data = extract_gmsh_geometry(gmsh.model)
    # Get mesh topology for each element
    topology_data = extract_gmsh_topology_and_markers(gmsh.model)

The topology data is a dictionary, where the key is the gmsh cell type (an integer). Each key accesses a dictionary with the topology data and corresponding topology markers. As this mesh only contains one cell type (triangles), as we did not mark any facets, we do not need to loop over the keys of this dictionary, only extract the first one.

In [6]:
if MPI.COMM_WORLD.rank == 0:
    # Extract the cell type and number of nodes per cell and broadcast
    # it to the other processors 
    gmsh_cell_type = list(topology_data.keys())[0]    
    properties = gmsh.model.mesh.getElementProperties(gmsh_cell_type)
    name, dim, order, num_nodes, local_coords, _ = properties
    cells = topology_data[gmsh_cell_type]["topology"]
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([gmsh_cell_type, num_nodes], root=0)
else:        
    cell_id, num_nodes = MPI.COMM_WORLD.bcast([None, None], root=0)
    cells, geometry_data = numpy.empty([0, num_nodes]), numpy.empty([0, gdim])

As we have now broadcasted all the information required to distribute the mesh in parallel

In [7]:
from dolfinx.cpp.io import perm_gmsh
from dolfinx.cpp.mesh import to_type
from dolfinx.mesh import create_mesh

# Permute topology data from MSH-ordering to dolfinx-ordering
ufl_domain = ufl_mesh_from_gmsh(cell_id, gdim)
gmsh_cell_perm = perm_gmsh(to_type(str(ufl_domain.ufl_cell())), num_nodes)
cells = cells[:, gmsh_cell_perm]

# Create distributed mesh
mesh = create_mesh(MPI.COMM_WORLD, cells, geometry_data[:, :gdim], ufl_domain)


We define the function space as in the previous tutorial

In [8]:
import dolfinx
from petsc4py import PETSc
V = dolfinx.FunctionSpace(mesh, ("CG", 1))

## Defining the load
Authors: Hans Petter Langtangen and Anders Logg.

Adapted to dolfin-X: Jørgen S. Dokken

The right hand side pressure function is represented using 
Python-classes. The class will have an initializer, where we store the two physical parameters, and an evaluation function which has a physical coordinate `x` as input.

In [9]:
import numpy as np
class Pressure():
    def __init__(self, beta, R0):
        self.beta = beta
        self.R0 = R0
    def __call__(self, x):
        return 4*np.exp(-self.beta**2*(x[0]**2+(x[1]-self.R0)**2))
pressure = Pressure(8, 0.6)

To be able to use this class in a variational formulation, we interpolate this function into the appropriate function space. In this case, we will use the same function space as we use for our approximate solution

In [10]:
p = dolfinx.Function(V)
p.interpolate(pressure)
p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)


We can modify the parameters at any time by changing the `pressure`-variables and update the interpolation, as follows

In [11]:
pressure.beta = 12
pressure.R0 = 0.3
p.interpolate(pressure)
p.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

We create the homogenuous boundary condition over the whole boundary which we do as in the first tutorial.

In [12]:
uD = dolfinx.Function(V)
uD.vector.set(0)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.full(x.shape[1], True, dtype=np.bool))
bc = dolfinx.DirichletBC(uD, dolfinx.fem.locate_dofs_topological(V, fdim, boundary_facets))

## Defining the variational problem
The variational problem is the same as in our first Poisson problem, where `f` is replaced by `p`.

In [13]:
import ufl
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
a = ufl.dot(ufl.grad(u), ufl.grad(v))*ufl.dx
L = p*v*ufl.dx
uh = dolfinx.Function(V)
dolfinx.solve(a==L, uh, bc)

## Plotting the solution
As mentioned in the previous section, we can use Paraview to visualize the solution.

In [14]:
import dolfinx.io
p.name = "Load"
uh.name = "Deflection"
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, "results_membrane.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_function(p)
    xdmf.write_function(uh)

Note that we gave the functions `p` and `uh` new names, to make them easier to identitify in Paraview. We start by opening Paraview. We start by opening the file by pressing `File->Open`.
The next step is to visualize each of the functions. To do this, we choose `Filters->Alphabetic->ExtractBlock`. The next step is to select the first Unstructured Grid and press `Apply` as shown below:

![Select Block](select_block.png)

We can select the other block to visualize the deflection. There are also options to visualize the deflection in three dimensions using `Filters->Alphabetical->Warp By Scalar`, and change the layout to 3D by pressing the `2D`-button.

![Change interactive layout](2Dto3D.png)

Finally, press the `Set view direction button`.

![Set view direction](axis.png)

With these instructions, you can obtain the following figures

![Results for membrane](result_membrane.png)